In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import healpy as hp
import numpy as np
import h5py, os

from msfm.utils import files, catalog

### global constants

In [2]:
conf = files.load_config( "../configs/v16/default.yaml")

n_side = conf["analysis"]["n_side"]
n_pix = conf["analysis"]["n_pix"]
z_bins = conf["survey"]["metacal"]["z_bins"]

# relative to this notebook
pixels_file = "../" + conf["files"]["pixels"]
noise_file = "../" + conf["files"]["noise"]

# Load relevant properties from Metacalibration catalog

In [3]:
e1_tomo, e2_tomo, w_tomo = catalog.get_shapes_from_cat(conf)

26-02-12 00:32:32   catalog.py INF   #gals in metacal bin 1: 24940465                  
26-02-12 00:32:34   catalog.py INF   #gals in metacal bin 2: 25280405 1s/it]           
26-02-12 00:32:36   catalog.py INF   #gals in metacal bin 3: 24891859 1s/it]           
26-02-12 00:32:37   catalog.py INF   #gals in metacal bin 4: 25091297 1s/it]           
100%|████████████████████████████|   | 4/4 [00:06<00:00,  1.71s/it]           


# checks

In [4]:
# check that we have zero means
for i, (e1, e2, w) in enumerate(zip(e1_tomo, e2_tomo, w_tomo)):
    print(f"metacal bin{i}")
    e1_mean = np.sum(e1*w)/np.sum(w)
    e2_mean = np.sum(e2*w)/np.sum(w)
    print("Mean e1: ", e1_mean)
    print("Mean e2: ", e2_mean)

metacal bin0
Mean e1:  0.00032158990997892836
Mean e2:  0.00016042357892342616
metacal bin1
Mean e1:  0.00033608292279748103
Mean e2:  3.735112133447201e-05
metacal bin2
Mean e1:  0.00037650112122322986
Mean e2:  8.747232497401608e-06
metacal bin3
Mean e1:  0.0004062292766666156
Mean e2:  -2.6810162966060493e-05


# create the noise file

In [5]:
with h5py.File(noise_file, "w") as f:
    for z_bin, e1, e2, w in zip(z_bins, e1_tomo, e2_tomo, w_tomo):
        # shape (n_gals, 3)
        sample = np.stack([e1, e2, w], axis=1)

        # save as np.float32, single precision
        dset = f.create_dataset(name=f"{z_bin}/cat", data=sample, dtype="f")
        dset.attrs["info"] = "This dataset contains the ellipticities of the Metacalibration galaxy sample per redshift bin. " \
                             "The shape is (n_gals, 3), where axis 1 contains (e1, e2, w)."
